# Exponential Map

In [1]:
import numpy as np

from sympy.algebras.quaternion import Quaternion

from scipy.linalg import expm, logm

In [2]:
# different brackets

def C(x,y):
    return x@y - y@x

def Q(x,y):
    return x*y - y*x

def R(x,y):
    return np.cross(x,y)-np.cross(y,x)

# initial terms of Baker-Campbell-Hausdorff
def baker(x,y,B):
    return x+y + B(x,y)/2 + B(x,B(x,y))/12 + B(y,B(x,y))/12

deg = np.pi/180

In [3]:
np.set_printoptions(precision=3, suppress=True)

## Simple example in Aff(1)

An element of the group:

In [4]:
m1 = np.array( [[1,2],[0,1]] )
m1

array([[1, 2],
       [0, 1]])

The corresponding element of the Algebra:

In [5]:
t1 = logm(m1)
t1

array([[0., 2.],
       [0., 0.]])

Another element:

In [6]:
m2 = np.array( [[4,3],[0,1]] )
t2 = logm(m2)

Composition in the group:

In [7]:
m1 @ m2

array([[4, 5],
       [0, 1]])

The exponential property $e^Ae^B = e^{A+B}$ fails because the group is not commutative.

In [8]:
expm(t1+t2)

array([[4.   , 7.328],
       [0.   , 1.   ]])

The Baker-campbell-Hausdorff formula gives a good approximation:

In [9]:
expm(baker(t1,t2,C))

array([[4.   , 3.635],
       [0.   , 1.   ]])

In the other direction:

In [10]:
m2 @ m1

array([[ 4, 11],
       [ 0,  1]])

In [11]:
expm(baker(t2,t1,C))

array([[ 4.   , 11.021],
       [ 0.   ,  1.   ]])

## Example in SO(3)

In [12]:
def rot3(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[c, -s, 0]
                  ,[s,  c, 0]
                  ,[0,  0, 1]])

def rot1(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[1, 0,  0]
                  ,[0, c, -s]
                  ,[0, s,  c]])

def rot2(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[ c, 0, s]
                  ,[ 0, 1, 0]
                  ,[-s, 0, c]])

We repeat the same experiment in the group or 3D rotations:

In [13]:
m1 = rot3(30*deg)
t1 = logm(m1)

m2 = rot1(40*deg)
t2 = logm(m2)

In [14]:
m1

array([[ 0.866, -0.5  ,  0.   ],
       [ 0.5  ,  0.866,  0.   ],
       [ 0.   ,  0.   ,  1.   ]])

In [15]:
t1

array([[-0.   , -0.524,  0.   ],
       [ 0.524, -0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ]])

The matrix logarithm of the rotation gives the angle (see the last section).

In [16]:
t1/deg

array([[ -0., -30.,   0.],
       [ 30.,  -0.,   0.],
       [  0.,   0.,   0.]])

In [17]:
m1 @ m2

array([[ 0.866, -0.383,  0.321],
       [ 0.5  ,  0.663, -0.557],
       [ 0.   ,  0.643,  0.766]])

In [18]:
expm(t1+t2)

array([[ 0.871, -0.46 ,  0.171],
       [ 0.46 ,  0.643, -0.613],
       [ 0.171,  0.613,  0.771]])

Again, we need the BCH formula to get a good approximation in the tangent space:

In [19]:
expm(baker(t1,t2,C))

array([[ 0.846, -0.417,  0.333],
       [ 0.534,  0.643, -0.549],
       [ 0.014,  0.642,  0.767]])

In [20]:
m2 @ m1

array([[ 0.866, -0.5  ,  0.   ],
       [ 0.383,  0.663, -0.643],
       [ 0.321,  0.557,  0.766]])

In [21]:
expm(baker(t2,t1,C))

array([[ 0.866, -0.499,  0.008],
       [ 0.377,  0.643, -0.666],
       [ 0.327,  0.58 ,  0.746]])

We now check the represention of rotations by unit quaternions. First we define two elements and verify that they correspond to the expected matrices.

In [22]:
u1 = np.array([0,0,1])
a1 = 30*deg
u2 = np.array([1,0,0])
a2 = 40*deg

q1 = Quaternion(np.cos(a1/2),*(np.sin(a1/2)*u1))
q2 = Quaternion(np.cos(a2/2),*(np.sin(a2/2)*u2))

In [23]:
q1

0.965925826289068 + 0.0*i + 0.0*j + 0.258819045102521*k

In [24]:
q1.to_axis_angle()

((0, 0, 1.00000000000000), 0.523598775598299)

In [25]:
q1.to_rotation_matrix()

Matrix([
[0.866025403784439,              -0.5,   0],
[              0.5, 0.866025403784439,   0],
[                0,                 0, 1.0]])

In [26]:
m1

array([[ 0.866, -0.5  ,  0.   ],
       [ 0.5  ,  0.866,  0.   ],
       [ 0.   ,  0.   ,  1.   ]])

In [27]:
q2

0.939692620785908 + 0.342020143325669*i + 0.0*j + 0.0*k

In [28]:
q2.to_axis_angle()

((1.00000000000000, 0, 0), 0.698131700797732)

In [29]:
q2.to_rotation_matrix()

Matrix([
[1.0,                 0,                  0],
[  0, 0.766044443118978, -0.642787609686539],
[  0, 0.642787609686539,  0.766044443118978]])

In [30]:
m2

array([[ 1.   ,  0.   ,  0.   ],
       [ 0.   ,  0.766, -0.643],
       [ 0.   ,  0.643,  0.766]])

Then we create the composition $q_2q_1$ and obtain the rotation matrix by conjugation of the basis:

In [31]:
q = q2*q1

#np.array([(q*Quaternion(0,*b)*q.inverse()).args[1:] for b in np.eye(3)]).T.astype(float)

In [32]:
m2@m1

array([[ 0.866, -0.5  ,  0.   ],
       [ 0.383,  0.663, -0.643],
       [ 0.321,  0.557,  0.766]])

In [33]:
q.to_rotation_matrix()

Matrix([
[0.866025403784439,              -0.5, 2.77555756156289e-17],
[0.383022221559489, 0.663413948168939,   -0.642787609686539],
[ 0.32139380484327, 0.556670399226419,    0.766044443118978]])

Now check the approximation in the tangent space:

In [34]:
tq1=Quaternion(0,0,0,a1/2)
tq2=Quaternion(0,a2/2,0,0)

First we check that the elements of the algebra, which are pure imaginary quaternions, are correct (it seems that the quaternion logarithm is not implemented). This is the quaternion exponential:

In [35]:
tq1.exp()

0.965925826289068 + 0*i + 0*j + 0.258819045102521*k

Then check the BCH formula:

In [36]:
baker(tq2,tq1,Q)

0 + 0.357040715800178*i + (-0.0913852259360126)*j + 0.251166233930734*k

In [37]:
baker(tq2,tq1,Q).exp()

0.902180815665869 + 0.345321162680651*i + (-0.0883855848242228)*j + 0.242921919234619*k

We get the expected good approximation:

In [38]:
baker(tq2,tq1,Q).exp().to_rotation_matrix()

Matrix([
[0.866353859101299, -0.499361816307834, 0.00829260116178332],
[0.377276164644994,  0.643484471520505,   -0.66602584822418],
[0.327251717201072,  0.580142664631483,   0.745882566000327]])

In [39]:
m2@m1

array([[ 0.866, -0.5  ,  0.   ],
       [ 0.383,  0.663, -0.643],
       [ 0.321,  0.557,  0.766]])

Finally, we check that the Lie Bracket is equivalent to the standard cross product:

In [40]:
v1 = np.array(tq1.args[1:]).astype(float)
v2 = np.array(tq2.args[1:]).astype(float)

In [41]:
baker(v2,v1,R)

array([ 0.357, -0.091,  0.251])

In [42]:
baker(tq2,tq1,Q)

0 + 0.357040715800178*i + (-0.0913852259360126)*j + 0.251166233930734*k

## Exponential and logarithm of rotations

In [43]:
import sympy as sym

I = sym.I
def mat(x1,x2,x3,x4):
    return sym.Matrix([[x1,x2],[x3,x4]])

theta = sym.Symbol('theta',Real=True)
m = mat(0,-theta,theta,0)
m

Matrix([
[    0, -theta],
[theta,      0]])

In [44]:
m.exp()

Matrix([
[     exp(I*theta)/2 + exp(-I*theta)/2, I*exp(I*theta)/2 - I*exp(-I*theta)/2],
[-I*exp(I*theta)/2 + I*exp(-I*theta)/2,     exp(I*theta)/2 + exp(-I*theta)/2]])

We explicitly compute the matrix exponential from the factorization.

In [45]:
m.eigenvects()

[(-I*theta,
  1,
  [Matrix([
   [-I],
   [ 1]])]),
 (I*theta,
  1,
  [Matrix([
   [I],
   [1]])])]

In [46]:
mat(-I,I,1,1) * mat(-I*theta,0,0,I*theta)* mat(-I,I,1,1).inv()

Matrix([
[    0, -theta],
[theta,      0]])

In [47]:
r = mat(-I,I,1,1) * mat(sym.exp(-I*theta),0,0,sym.exp(I*theta))* mat(-I,I,1,1).inv()
r

Matrix([
[     exp(I*theta)/2 + exp(-I*theta)/2, I*exp(I*theta)/2 - I*exp(-I*theta)/2],
[-I*exp(I*theta)/2 + I*exp(-I*theta)/2,     exp(I*theta)/2 + exp(-I*theta)/2]])

In [48]:
sym.simplify(r)

Matrix([
[cos(theta), -sin(theta)],
[sin(theta),  cos(theta)]])